[View in Colaboratory](https://colab.research.google.com/github/Bravest-Ptt/Useful-Shell/blob/master/How_Many_People_You_Connect_In_WeChat_A.ipynb)

 ### 声明：<br>
此程序仅用「个人微信数据统计」，「程序本身」**不**上传或「偷偷收集」任何您的信息或数据，不信？看源码喔。<br>
程序是在「光明正大」的在「你的监督」下，对你的微信联系人和群聊联系人作一次简单数据统计和分析。<br>
但运行在「**Google Colab**」服务中的程序是否存在「数据泄露」暂不清楚，就看你信不信Google了。<br>
* Google Colab分配的服务器资源会在一定时间后回收
* 从目前来看，回收后，以前数据都会被清空<br>

如果您选择执行此程序，则代表您**愿意**「接受此风险」并认同此风险与此程序及其开发者无责任相关。

---



## 你的微信，到底「连接」多少人？


### 执行时间
大约5分钟

### 边学、边看、边生成版
* 此版本是明确进行步骤划分的，可以通过一步一执行来体验
* 点击代码左侧的开始按钮，即可开始一个代码段

### 如果你想统计群聊人数，需要手动在手机端将群聊加入通信录。

## 准备

### 安装itchat依赖

In [0]:
# Colab/Jupyter 中的安装方式
# 安装itchat
!pip install itchat
# 二维码生成显示
!pip install pyqrcode
# 汉字转拼音
!pip install pypinyin
# 进度条显示
!pip install tqdm

### 依赖库导入

In [0]:
# coding=utf8
import json
import time
import os
import base64

import itchat
import pyqrcode as QR
from tqdm import tqdm
from pypinyin import pinyin, lazy_pinyin, Style

# [linux] sudo apt-get install python-pandas
import pandas as pd

# [linux] sudo apt-get install python-numpy python-matplotlib
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image, ImageDraw, ImageFont

# 解决matplotlib图例无法显示中文
# import matplotlib
# print(matplotlib.matplotlib_fname())
# 查看输出配置文件位置 如：
# /usr/local/lib/python2.7/dist-packages/matplotlib/mpl-data/matplotlibrc
# 修改其中的 #font.family: sans-serif -> font.family: Microsoft YaHei
# 参考https://www.zhihu.com/question/25404709

### 定义全局变量


In [0]:
# 好友列表
all_friend_list = []
# 群聊好友数， 去重后
friends_in_chatrooms = 0
# 重复个数
duplicate = 0

# 防止qrcode多次回调的flag
has_return_qr = False

# 后续用到变量
base_path = './how_many_friends_in_your_wechat'
result_path = base_path + '/results'

title_path = result_path + '/0.png'
summary_path = result_path + '/1.png'
shixoong_qr_path = result_path + '/999.png'

user_qrcode_name = 'qrcode.png'
user_header = base_path + '/user_header.png'
user_nickname = ''

### Friend类定义

In [0]:
class Friend(object):

  def __init__(self, user_name, sex=0, province='', city='', remark=0, star=0):
    """
    构造方法，初始化Friend类
    :param user_name: 每次登陆后，系统为所有相关人生成的「一次性」唯一标识
    :param sex: 1为男，2为女，0为unknown
    :param province: 省份
    :param city: 城市
    :param remark: 是否备注 RemarkName
    :param star: 星标好友 StarFriend 0 不是，1是
    """
    self.user_name = user_name
    self.sex = sex
    self.province = province
    self.city = city
    self.remark = remark
    self.star = star

  def __eq__(self, other):
    """
    根据唯一标识UserName，判断两个Friend对象是否相等
    :param other: 另一个Friend对象
    :return: True即相等，False则不等
    """
    if other is None:
      return False
    if type(other) is not Friend:
      return False
    return self.user_name == other.user_name

  def __str__(self):
    return self.__dict__

# 用于解决Friend无法直接转换成json数据的问题
class FriendEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, Friend):
      return obj.__str__()
    return json.JSONEncoder.default(self, obj)

### 定义：合并图片

In [0]:
def combine_images(imgs_path, result_w=800, inner_img_w=700, inner_img_spacing=50, top_margin=30, bottom_margin=30, bk_color='white'):
  imgs_path = imgs_path
  width = inner_img_w
  result_width = result_w
  result_height = 0
  image_spacing = inner_img_spacing
  image_margin_top = top_margin
  image_margin_bottom = bottom_margin
  bk_color = bk_color

  imgs_h = []
  for path in imgs_path:
    im = Image.open(path)
    scale = float(im.size[0]) / im.size[1]
    h = int(width / scale)
    imgs_h.append(h)
    # print "path = %s, height = %d" % (path, h)

  result_height += sum(imgs_h) + (len(imgs_h)-1)*image_spacing + image_margin_top + image_margin_bottom
  # print result_height

  bk = Image.new('RGB', (result_width, result_height), bk_color)
  x_paste = (result_width - width) / 2
  for index, path in enumerate(imgs):
    # print "path = %s, index = %d" % (path, index)
    im = Image.open(path).resize((width, imgs_h[index]))
    # if is first picture y coordinate is 0
    # else equals to sum of previous images and sum of spacing
    y_paste = (image_margin_top if index == 0
               else (image_margin_top + sum(imgs_h[0:index - 1]) + imgs_h[index - 1] + image_spacing * index))
    # print y_paste
    bk.paste(im, (x_paste, y_paste))
  return bk

### 定义生成Summary

In [0]:
def generate_summary(all_friends, remarked, star):
  global summary_path, user_header
  width = 700
  im = Image.open(summary_path)
  fnt_path = '/usr/share/fonts/truetype/ttf-liberation/LiberationMono-Bold.ttf'
  color_all = "#000000"
  color_remark = "#000000"
  color_star = "#000000"
  
  # 支持调节1位到99w位的
  fnt_all_size = 100
  fnt_all_start = (340, 198)
  all_friends_len = len(str(all_friends))
  if all_friends_len == 1:
    fnt_all_start = (395, 168)#
    fnt_all_size = 120
  elif all_friends_len == 2:
    fnt_all_start = (370, 178)#
    fnt_all_size = 110
  elif all_friends_len == 3:
    fnt_all_start = (345, 182)#
    fnt_all_size = 100
  elif all_friends_len == 4:
    fnt_all_start = (340, 188)#
    fnt_all_size = 80
  elif all_friends_len == 5:
    fnt_all_start = (340, 198)#
    fnt_all_size = 65
  elif all_friends_len == 6:
    fnt_all_start = (340, 206)
    fnt_all_size = 50
  fnt_all = ImageFont.truetype( fnt_path,size=fnt_all_size)
  
  # 支持调节1位到4999位的
  fnt_remarked_size = 100
  fnt_remarked_start = (455, 308)
  remarked_len = len(str(remarked))
  if remarked_len == 1:
    fnt_remarked_size = 100
    fnt_remarked_start = (485, 292)#
  elif remarked_len == 2:
    fnt_remarked_size = 80
    fnt_remarked_start = (468, 300)#
  elif remarked_len == 3:
    fnt_remarked_size = 60
    fnt_remarked_start = (457, 310)#
  elif remarked_len == 4:
    fnt_remarked_size = 50
    fnt_remarked_start = (455, 318)#
  fnt_remarked = ImageFont.truetype(fnt_path, size=fnt_remarked_size)
  
  # 支持调节1-999位
  fnt_star_size = 70
  fnt_star_start = (450, 415)
  star_len = len(str(star))
  if star_len == 1:
    fnt_star_size = 90
    fnt_star_start = (474, 400)#
  elif star_len == 2:
    fnt_star_size = 70
    fnt_star_start = (455, 413)#
  elif star_len == 3:
    fnt_star_size = 50
    fnt_star_start = (450, 423)#
  fnt_star = ImageFont.truetype(fnt_path, size=fnt_star_size)
  
  draw = ImageDraw.Draw(im)
  #338-528,198
  draw.text(fnt_all_start, str(all_friends), fill=color_all, font=fnt_all)
  #455-569,305
  draw.text(fnt_remarked_start, str(remarked), fill=color_remark, font=fnt_remarked)
  #454 - 546, 415
  draw.text(fnt_star_start, str(star), fill=color_star, font=fnt_star)

  # paste avator
  # 130 * 130
  avator_w, avator_h = (130,130)
  avator_im = Image.open(user_header).resize((avator_w,avator_h))

  bigsize = (avator_im.size[0] * 3, avator_im.size[1] * 3)
  # 遮罩对象
  mask = Image.new('L', bigsize, 0)
  draw = ImageDraw.Draw(mask)
  draw.ellipse((0, 0) + bigsize, fill=255)
  mask = mask.resize(avator_im.size, Image.ANTIALIAS)
  avator_im.putalpha(mask)

  avator_margin = (width - avator_w) / 2
  im.paste(avator_im, (avator_margin, 0), avator_im)
  im.save(summary_path)

### 定义微信二维码显示方法

In [0]:
def qrcode_handle(uuid, status, qrcode):
  global has_return_qr, base_path, user_qrcode_name
  if has_return_qr:
    return
  img_file = base_path + '/' + user_qrcode_name
  img = open(img_file, 'w')
  img.write(qrcode)
  img.close()
  
  qr_img = Image.open(img_file)
  plt.figure("Image") # 图像窗口名称
  plt.imshow(qr_img)
  plt.axis('off') # 关掉坐标轴为 off
  plt.title('Scan QR to login') # 图像题目
  plt.show()
  has_return_qr = True

### 定义初始化函数
检查当前目录下以下内容：
 + 检查并新建文件夹：./how_many_friends_in_your_wechat
 + 创建结果存放文件夹：./how_many_friends_in_your_wechat/results/
 + 相关需要图片准备：如模板、水印。

In [0]:
def init():
  global base_path, result_path, title_path, summary_path, shixoong_qr_path
  # 在当前目录创建base文件夹
  # 判断是否已存在
  if os.path.isdir(base_path):
    # 删除已存在
    os.system("rm -rf " + '"' + base_path + '"')
  # 创建base文件夹
  os.mkdir(base_path)
  # 创建results文件夹
  os.mkdir(result_path)
  
  #将以base64编码存储的素材转化为png，放入results文件夹备用
  title_base64 = "iVBORw0KGgoAAAANSUhEUgAAATUAAABwCAYAAAB//edvAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsMAAA7DAcdvqGQAABblSURBVHhe7Z3Pi23HWob9exw5kQx0ElEnFwI6ESIoiKiDiCAXLnFwURQUIU6SDJKBoAMdXVQiBPEGD8SbkGAiREwMB/FHTsQQHSSQDJy05+nu9+Ttj6pVVWvX6t691vfAR/feq1b9/t76qtbu3T90kSRJsiNS1JIk2RUpakmS7IoUtSRJdkWKWpIkuyJFLUmSXZGiliTJrkhRS5JkVxRF7a3P37x49eFLZ2+ffv3JdY2TJEmuKIraCx/+/sWPvv7DZ2/v/s/b1zVOkiS5IkUtSZJdkaKWJMmuyDO1JEl2RVHUkiRJ7ispakmS7IoUtSRJdkVR1P7zq/+4+If/fffs7cv/+/K6xkmSJFcURS2ffiZJcl9JUUuSZFekqCVJsitS1JIk2RVFUUuSJLmvpKglSbIrUtSSJNkVKWpJkuyKfFDQwWuP/uKyvF975xev37l7GKNvv//r16/uhv/6+tHF73zwW5cfhD4nvv3ecxfPPPipi4+/+Oj6neRIpKg14K8WfuL7T12W96tnImqIiPrgth3Xy/uzf/uTyzogIOeE+oZxS2E7HilqDX77g+eflLdFRIJo/tFHf3AZ8bz68OWLP//3P2064jMPfvKyPrctspRHuXztEyiCxc4Jn78Ibv453bFIUVuA7ZXK2kpAXBjciDJKWzuP0h789/ev370Jaaj7bFQu8wO8LoAYIyKItEBQ2LbTni0WhRoSYK9vcgyKorbmD9p/7s1nnkwi7MWPX7hx/ff+6bs3rv/KO79w4/pfffK9G9cxv4799Bs/Hq6/c13j08DxEJCf/8HPXoqMwBlVlkSCtA8+e+MyquJMizQ4Mmn4yWuM6z1bH/LlDEjllIy2C0WORGtAGZRF3eN9UWBOgTooX4mpvwe8r9dqO1Gd3vO+nQFj8deP/vKy/Zj6Hiv1R80Yx4zm9kNR1NbwS28/e2OixAn8x//6yo3r33n/N66vXPHwy49vXP+xv/mR6yvf8K2/e/pGmhmRGs6gMzMMgQF3WJwEcRhxFBn39joM6SiXsyoXOomIR44IlsS0ZQj2qej8DJNgeR/xO/XXa9LzWn379N8+NUU4KEdoGz7DPN/kfnNYUWMSR5FCSOR4IwLG9kZfMU4kxdYHJ9Z1hG0N1MWjPRcWN8qiXK7TLoz+f/bvf+ZJGq71QrlEQIghwokweX8IytF7EgWVSR94lKZzuDUg6kRTyksLT4xwKROjLzQePaa6J/vgcKKGwxJ1eT6Igp9PubO64bAIGGn93LG2zXSnm3HG5flJyGplA22VyCBOS1A/HlIgwCqjZIiGKIma6ki5HgFrseiF+hBFl6LRPCNLljiUqCEA0Ulw0uhw7tis5EQ5UZTcoV0QHe5VmiXx6UV5IWa9MA66r1ZP3ncBckM8ERGPwERJ1Liu96LR90RciGdN5OknIsTS/eStcpKkxqFEreS40UncUZfEA6dUOsSrhItjC5xZB944Pj95ICHB5bryGnVs3VerZxQiRYEugkrDNlR4X/E7dayJY8niHKGttftnLArJMTiMqOEw3IPDegQVHd2FqBZNCKXTGY/jIlS6DtRJ51ZKWzL6UvXXU88RFGXVtm2IF6LF9Zpgep8hbPSTn3PV2qDzLfKn7/1a7HvvM9qpemMt6J/3HtedKFCLA3Us2awnwsl5cqhIjYmPgfJwx+qN0kRpSyZcBGJfCHdaGc5MfvEa4ofItoS2hKKsUj17odwoSj0W68tr+rkmngibojLvwxocDYw81JHVtuLJ/edwDwqERMOjKD+I7xEPiQUWwTG5vnSorXsRstK5HQLsdaqJY4s1okb9Fe2wJSQqo36Kukqmp6+qb21hoB08XW3holYbD12XIbyqD/eXbG0/JveDw4oak548+CkUiTDxBc7EtqYE6VSXUXyr1YoaJGyIiyLNEUptLUE9iAhr51o9ZbuolaIx5oWutxYO799aZEd+iCeCqggvOTaHFTUcgTw4CxI4dRQ0lYXTRHqcrkbL+R1PW6pHC4laLXJCrJa2cES1ve0jL6LTmlDrA7Pk2eKU/k2Oy2FFrSfK8jQlMRkRpgj56d4etF32aIsyW9EOqBwXbMfrIvMtMeUQwemgPaZ1I8qrleNRWpwfJRBGpU9RS3o5rKi5I9e2Vf5EryQeLmol0VvCBVNiIeMsK5anyFIfqfCzNs67+PhHDaWriQ1lIZYqI6bVe72GIJZQlFa7HvH+rYkaYxefetIfsU/duJ5b1f1yWFFrOQyOruu1rZLnwZZL0QwfGeAgvHYWBy5qJYtbRU8PlOfpsaso6eUbIu11bJ3dgdK6qFGWP/lEALkukeIarzFFd5HRKA166q6t9ah5xJvsixS1x1YSNY/klqIwpVmy0ueiFGkhDIgBr3E0RUwxkiCN8hMuLNHU/612RpRfdHoXeeqGcOqBQu2sztH2GQHsxetOW0u4qPU8+ZSVhDfZB4cVNVA+TPKIH5wvOYDSIEqYRzRucYsrZ4ziUYM6Kq8IUYxvRzEEB/y+2jbbqdVLkaG2jh55tSLAHnEq0XtfClTipKg9tugwHpW0ntKV8uB+2s97RDElp58paoJydS6mekvseiMkpdfZnVCkpahM6SRyQuWzDddn0dRWrEdYxYgYEj2y3de5Ws1axwLJ/efQoiZHxUEdj0JazqR0veIk5Py9942mF2rjKeLpIk/fIEx6TQTnuIBh/rBFgthLj6jxfuvPzGqWfy61Tw4tanLA6PC+lWs9JVO6UbEpiccS2tZGAW6hMlriLLxe2tYRafp7/hrhcRA87z83/anXCLq3VH8JvRsizlgsmR8RUKdkXxxa1OQU/gFcUP49WzalHRUbjwajMEQ8bezXJTzSaZ17Cb9H9ZLQqT96BBlxKz2hxUbETfcgRo7XEyFr9WHEBXFUaJPz5tCiVnJOd5ae7ZLSliKJJXB6RQycX9UcCzHSU8YekXW8fb2OS2Sqe9QmRV4SFuUbhSbi5Xt0hNGmVhQMui8uGp73yDmdoD90/8hCkZw/hxY1CZiLhTtLz2RXWgnACOSv+zEO13Wgrc+8+fVR50N0uC8e5rdQeTovU8Sl8r2PqCtnU9QVoeKDrUIfD/EnpnoPa4kiIOosLlEAvQ5rcFEb/eB0ct4cWtQAZ/GtC5OdqKBHpDyqGRUcgUPFKCYa13u3j47u74k4ndLDBY/0XBCicS/4uVvsG16T99o+A/8c4WikdjnGj8VX9/dEjMn94fCiNgoOpL8YIJpSXdZsgQT34qSID86OiPETcaUf1+R9iuCSHnFaimAQLUVdqi/pJX7asnLtlL6p4e3jTJSxUJSLxT+dwhRN6j5MIpzshxS1QUqH36OR0G2gaGorUWlBuZQfP/IxE+aYj8OIUT/qdhd9k2xLitogvq3CWOnP1TEQtruqG/3EFn7r8smfCJFIEfMzuzhOXEfIqFuK2X5JUVsBYsE5XJ7FJMn5kaKWJMmuSFFLkmRXpKglSbIrUtSSJNkVKWpJkuyKFLUkSXZFilqSJLsiRS1Jkl2RopYkya5IUUuSZFekqCWHgz9x45s9+NYOfatIsh9S1JLp8NVMfF/Zuf7RuL4WCev9Bt7k/pCilkzFv2OOb8M4R/wLJjG+Tj2/tWM/pKgl02DMNTZ8X9m5bu0QsPgVRYgxXywZjS8D1ZdO5lb1fpCilkzBv4kWW/M/G2aCAOnbbkvfeLvWlr4NODkPUtSSbhAuIpZ4BkXkwxZO43IOX5G99h8ctyz+V6vk/EhRS7qoiQQRkAsaxkMCtm3aws0+ryIKoz5LZ2F8xbrXSca2U9+SSzRZM9+eItK8R5SWZ2/nT4pa0kXta7JHTdtB/0cpo8LnB/1L20EiSj6+4f9Xouf/SSCaSo+d6wOPpEyK2j0Ah+c/I/m/8rttqAMCQsTCFmzp/wGsMSK+3vZRB93H7y3IV+mJ8Fp4/rRxNogmwu7/I/UuoF9YXPb2kZYUtY2YuU1xJzun7Y/XCyOiwVF4H9O//MP036WWbI2o9Z5xeTmtp5j+UGH2gwH8wvNHVPxhhj4UzCK29VgzLtSBn3vikKKG82kilcy3Rm46I2qh85xeJ23h2y0cejYPPnvjyRlYLzicO+dIvbQtlDEeI9GC90evQ3IupnuWhIq6KN3Mfy9IPswr5d1jnBluiffjXS6WOiNlPs2oxyFFzSf4GmtFB8q/5egMoD4HVTOEBtFR2Qy8xDXaGjzqwZhc1KmFH8SzRYuTkTq3IqK1IIQqu1fU/Fxtafy4pnQ95289INj+MIX+ot4Y/Y8pysW8rrMjRYd6qRzKvyvcH2dwSFHD2ZhITB5NLs+3tVVqTTTl1xI1Xym3MASqFQG5OLktPVmk/zxtdAjGXtdmCBttIIL2MmWIfIy0EYaIR2DcUyK2q5TPKJTrES393RONSFx7t9dr0bkovnAX+KI6q615pvYYd+wZq3OvqPlKyWrFfT1nTyPWEmAcjDaXyq1tf1yMqXPEJ2qrD3qoCe+SReHgtV8vCZa3a8YDAhd3bGRuKXqZMR+XUN+WxnFrGAP1DfNvVmR/eFHziTdrAknUeiaKyh6JCq4izZcv/3CcyISzGt++uPWC0+PUUdxKefgWrbRtmd2niP/IkYGv+GzLtZX3iEkH8jJeex7RSDMi0BILN8qnHkswDpRF+hmOzlhQZhR54ULu0FaeznLv2qONJaiP9/mMxU8cWtQ8UlqzUuHQsZ3QK2oIhsp38SBPHd4zqfSEbMnxWhFZL/SJtiQ1p1KZXC8R+5XXPM3DSbwNI6IbUR7KR+b19bOpWUZblsBZ/YEAfeSRK0YEXOpX36pyX6usFuSnMkvzFOgzpRHUTe+5XQn7yyfXC3xsWLBmcmhR0+pfOugGBo+BrDmf6hEnqCKZEVETvnL2GG2gvNpKvBbqVstTZZfOpriHld3rWLNThNjzqdWzFC2NGmMo64k6vUzGRnWjPxEqXaPvFOUyfxB8v+9U4aBcCWRtfoPPQZ/n/E5btMBFY3FaW0cvE5shks5hRa02mI6vsKWVrna/7lsjatyj92RMct4nX4RAjrP1I/8aPtGZ3Ag/JidqGe2hHTVH64E8lF9t/MifcVO/kQ6b7USOxo8xiu3jtW/dMaI67zciGN1HPdd+Xs3LqfUPeFRWS0c96D/ydGGuRX8tfOwYm9kcVtQU/tLBNZhMPgDx/Kj2fq+oMVGUB1CeXi85PX2rdKVtzNZ4vXsMEcTJqfcaBy3h4r/ktA7bebb1W8J4tJyd/nNxUB+pHeThW9hR8fBtZ89TTaXt7Ufqt3ZhkG9gS753CocVNV9Rl/BBYEX1wdT7cbXRPa1IKoqfT8YlXPxGJ3wN8hz5UyzKdWHBeI3Rp34t9s8MPH/qzVkPWzgiRn5GsfcznLgIleB+8p0d1ZEfdVRddOYGUcywpa1jCdLq3JJ7W/hc6h37tdA+lUW74xjN4vCRGkLlg0lHs5rHbYGM9zTJFMXVRA1bIoqa6qTXS6hstgQCh+E8Sw8Yej5EK7SlxUacqIYLdGvhGIUxYsFQ/iXzMWF8/VpL1Gi/8kcgZkCeUbAYO9pSuobT8x7jOSKsPvd6RMr7Zmu2XujEYUWNyeRbgJKAyRAQX+klJBqkOEBxW1kjipjy431FHzImuM6uMH+KuGS9q6+3f8YKiiMqv6WItffjAuTHR1h62602IBjetp4Fw8ePOXIKlM/4eR201Sxdqxnt7okwNad9sYtQthY8/JT0p7azhffpVttOcVhRAxzFVw833mcg3ME1AXgfdG+cQL2rn4uYv2bS6/41xgQlLyIkHKeFC1DPlqUXrxMgYIg1Au2RFk69JKTenzVTm1lgEArKwmIEhDPrmiyW7cJ5SpTJfHHBoo6aO/Faj7X6Cch/adx5v5R3q52UOxL5O5TpbR2JPNdwaFETDBiOw8CSLwPQQ227OCpqivS0ypIvdeF6NAT0ynFferIFpb6U2SNgJSTW2ClOTPkSCsRrxGmXJrqv8jL1FeaRsp9XzTDEV9ExYtnr2N6nGHWkf2hnrKPGkQWF/ude7w/uU3sliqfAHIrl1+YO7xMhKy3zbBT5FaYFfEtS1Izeg3rBRCUtYuOsFTXd0ztxvZzY3yOoHZgLRAlFWnJ0bES83OTELUfBsegT6qa0LnTuKOrTLa2nrzWXEBAWTdoQo0ZtQ3uIc+VUEE3quFQ+1zxqpb418atBHrpfxoOcLZ9Cp6gZDLTyXoochMQgnhn5QC5NAq3QmqhajUcmriY7k290wglfSZcmOWOqdKNG26grbcOZWtuoFt7H5CtcAL18+pp0OKa/P2q0Y7Tu1CcK/0jEwlzU/acsXr3Qh/5hYNV3zfzSHC8Zc5bo99S5EElRu4aJw7ZJebOq0uEyVpY4qB4tOO5wSyKhNOQDEqiRCc+EkBhS5zWoXGypvpSldCWTcGmLTDt0zYVnBjVRE75AYT0O6eJOnpQRbRTPE9NYIVK9EZ8EbWmbOIsowCPRZMT9gzlBe+PWV0bEv/bMLnJYUWNyIAIeXrcsRlAMtq457vxLEyKm0YAzqUYmr0+eNcLmE63laNSL+sqWVlnSKl/6eSaed0nUqNvS9Yjnh2lMTkUiRh3oK/rXy2E7X4K0fva2taDF8rC10RmQn8QxRrf8zpwtRXHcwxmepx/lsKIWtyfRFHW4xYnujhPR+0vOEdN4dMGWNk4oBlpnWnF77BEB95YiyxreF1G4T0VOjc12Sm0lGZuIC31P5OvpyXcWjG30hShsvhAxxrz26yw6Wwqatx07JToTvlAuzSnmMXPX54lsbfR26EiNzsb8jEcOXnKUiItQFBm9PyJq4Fs2jAindBhfmihxq4NxNtLCJ3VPu0cgP+V9qqNEyI82x74HbxNpWnjEHufuFlBnd2SiE4459BpDXJibW4EPxLMz+upUAWVcvA290O8+X2TM/ZGztzxTC8gZ6MgeNDFjh6uOo6IGceWMxipYcmQgL58YPZOKunv+tbzX4G3Z0kEjtEHlIlhLbaJeSst4RjhrJWLAZkZM1Km0BaMOWwsr88SFnDJnjc+pCxnzsRa9lXYpkRS1AIOg/HsmMOlLE1B5LE0UpSkNPAPLvYomMdKNONXIhLophOufpJZA2EqR5dYg6GoTixRRCdt3fZZO5mdJ/rk0vec28hBnCRwzbjNxYvppZt+XoAwvl7GfVaYvJuR7CtQJ3/JxlNF3tcgtRS3gUQuigBH6MsnZIvSiPJac2cvphUkTndKttYrVoA6qD4YAbO1cW0NfLH2kYI0h0KdAP0fBxGm3jsyAue1lI6Kntifigrl2Lpagf3zhxWq7qcOLmouEPr7hZUTrdXQNQE3UXESiqBGhISpMQN8i9FrPGVKJeJ7HpGFFpG+W2s01HkxI/HX/bThqC+pW28r0GILDWDKOI4tPhL6IY3lbYgbMKcZTZSP2tUjnFDTvZ0W0EcZADyFqkeBhRY0BHREMOnBkUrdEzSPCuKKtdUDZKasvAlDKU6btGT9L192YfOeEb+epm8ZIRtslXtiMSJU84jwbnUsz8DlVm5MzYC7Pjv5GObSoeV4yBp9Jx8DThrUhtKKepZWYNKUVjQkv58OYJHI02ZbQ5pa4tYw+XNt3t4nqS9S0BT7P6NO76hMiM9p4H8bkVA69/WTCSST4fSas0FuLz9bQJ4gqArV0NqWFAIEmGpoR4dwWRGxbOzv9eJ/65L4zTdT+8J9/93Jiy976/M3rK1e8/ulrN66/8vDF6ytXPPrqkxvXn3v3l6+vfMPz//ibN9L8yxcfXl9JbguJ9exFIElmMU3UkiRJzoEUtSRJdkWKWpIkuyJFLUmSXZGiliTJrkhRS5JkV6SoJUmyK1LUkiTZERcX/w9SUadsiLngZQAAAABJRU5ErkJggg=="
  shixoong_base64 = "..."
  summary_base64 = "..."
  
  with open(title_path, 'w') as f1:
    file_data = base64.b64decode(title_base64)
    f1.write(file_data)
    f1.close()
  
  with open(summary_path, 'w') as f3:
    summary_data = base64.b64decode(summary_base64)
    f3.write(summary_data)
    f3.close()
  
  with open(shixoong_qr_path, 'w') as f2:
    sx_qr_data = base64.b64decode(shixoong_base64)
    f2.write(sx_qr_data)
    f2.close()

#init()
#!ls how_many_friends_in_your_wechat
#!ls how_many_friends_in_your_wechat/results/
#!pwd

## 扫码登录微信

In [0]:
init()

if itchat.check_login():
    itchat.logout()
    
# 登录微信，需要扫面下方输出结果的二维码
itchat.auto_login(enableCmdQR=True, qrCallback=qrcode_handle)


### 获取数据并去重
* 联系人列表
* 已保存到通讯录中群聊的人员

In [0]:
# 获取当前好友列表
friends_list = itchat.get_friends(update=True)

# 获取头像并保存本地待用
user = friends_list[0]
user.get_head_image(user_header)
# 获取昵称
user_nickname = user.NickName

# 好友个数
friends_num = len(friends_list[1:])
# 备注好友
remarked_friends = 0
# 星标好友
star_friends = 0
# 将好友列表加入最终朋友列表，去除自己
for one in friends_list[1:]:
  remark = 0
  star = 0

  # 备注
  if one.RemarkName and one.RemarkName is not "":
    remark = 1
    remarked_friends += 1

  # 星标
  if one.StarFriend and one.StarFriend == 1:
    star = 1
    star_friends += 1

  friend = Friend(one.UserName, one.Sex, one.Province, one.City, remark, star)
  all_friend_list.append(friend)


# 获取当前所有群聊列表
chat_room_list = itchat.get_chatrooms(update=True)

# 获取微信群聊中的所有成员
if chat_room_list is None:
  print("你连一个群聊都没有找到~")
else:
  print "共找到群聊：%d个\n" % len(chat_room_list)
  for room in tqdm(chat_room_list):
    room_updated = itchat.update_chatroom(room.UserName, detailedMember=True)
    # print "正在处理:%d/%d，请稍后" % ((index + 1), len(chat_room_list)), '\r',

    # 获取一个群聊中，所有群成员的公开信息
    #print "memberlist = %d/%d" % (len(room_updated.MemberList),len(chat_room_list)), '\r',
    for member in room_updated.MemberList:
        friend = Friend(member.UserName, member.Sex, member.Province, member.City)
        if friend not in all_friend_list:
            all_friend_list.append(friend)
            friends_in_chatrooms += 1
        else:
            duplicate += 1
    # break # for test

print "\n连接了%d人, 群聊人数%d, 好友人数%d." % (len(all_friend_list), friends_in_chatrooms, friends_num)

### 绘制数据

In [0]:
# 初始化表格元组数据
data_json = json.dumps(all_friend_list, cls=FriendEncoder)
df = pd.read_json(data_json, orient="records")

# ------------------------------------------------------------------------------
# 获得好友数量，去除自己
# friends_num = len(friends_list) - 1
# print friends_num

# ------------------------------------------------------------------------------
# 获得熟识好友数量,占好友比例
# remark_num = len(df[df['remark'] == 1])
# print remark_num
# remark_rate = float(remark_num)/friends_num
# remark_rate = '%.1f%%' % (remark_rate * 100)
# print remark_rate

# ------------------------------------------------------------------------------
# 获得星标好友数量
# star_num = len(df[df['star'] == 1])
# print star_num

# 开始画图
# 配置显示字体大小
# 默认{‘fontsize’: rcParams[‘axes.titlesize’],
#‘fontweight’ : rcParams[‘axes.titleweight’],
#‘verticalalignment’: ‘baseline’,
#‘horizontalalignment’: loc}

font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 24}
matplotlib.rc('font', **font)

# 这行代码使用「手绘风格图片」，有兴趣小伙伴可以google搜索「xkcd」，有好玩的。
plt.xkcd()
bar_color = ('#55A868', '#4C72B0', '#C44E52', '#8172B2', '#CCB974', '#64B5CD')
title_font_size = 'x-large'
  
# ------------------------------------------------------------------------------

# 获取性别数据
male_number = len(df[df['sex'] == 1])
female_number = len(df[df['sex'] == 2])
unknown_sex_number = len(df[df['sex'] == 0])
sex_data = [male_number, female_number, unknown_sex_number]

# 绘制「性别分布」柱状图
# 'steelblue'
plt.bar(range(3), sex_data, align='center', color=bar_color, alpha=0.8)
# 添加轴标签
plt.ylabel(u'Number')
# 添加标题
plt.title(u'Male/Female in your Wechat', fontsize=title_font_size)
# 添加刻度标签
plt.xticks(range(3), [u'Male', u'Female', u'UnKnown'])
# 设置Y轴的刻度范围
max_num =  max(male_number, max(female_number, unknown_sex_number))
plt.ylim([0, max_num * 1.1])

# 为每个条形图添加数值标签
for x, y in enumerate(sex_data):
  plt.text(x, y + 5, y, ha='center')

# 保存图片
plt.savefig(result_path+'/2.png')

# 显示图形
plt.show()

# ------------------------------------------------------------------------------
# 统计省份数据
# 省份名称集合
provinces = []
# 各省份人数
provinces_people = []
# 饼图中哪个突出
provinces_explode = [0.1]

# 获得省份数据副本,已按数量排序
province_df = df['province'].value_counts().copy()

# 丢弃其中省份为空的好友数据
for p in province_df.keys():
  if len(provinces) >= 6:
    break
  if not p:
    continue
  # 转为拼音
  provinces.append("".join(lazy_pinyin(p)).title())
  provinces_people.append(province_df[p])
  provinces_explode.append(0)

# 删除多余的一个0
provinces_explode.pop()

def make_autopct(values):
  def my_autopct(pct):
    total = sum(values)
    val = int(round(pct*total/100.0))
    return '{v:d}'.format(p=pct,v=val)
  return my_autopct

province_fig, ax1 = plt.subplots()
ax1.pie(provinces_people, explode=provinces_explode, colors=bar_color, labels=provinces,
        autopct=make_autopct(provinces_people), shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.title('Top 6 Provinces of your friends distributed', fontsize=title_font_size) # 图像题目
plt.savefig(result_path+'/3.png')
plt.show()

# ------------------------------------------------------------------------------
# 进行城市数据显示
# 城市
cities = []
# 各城市对应人数
city_people = []

city_df = df['city'].value_counts().copy()

# 丢弃其中城市为空的好友数据城
for c in city_df.keys():
  if len(cities) >= 5:
    break
  if not c:
    continue
  # 转为拼音
  cities.append("".join(lazy_pinyin(c)).title())
  city_people.append(city_df[c])

# 绘制「性别分布」柱状图
plt.barh(range(len(cities)), city_people, align='center', color=bar_color, alpha=0.8)
# 添加轴标签
plt.xlabel(u'Number of People')
# 添加标题
plt.title(u'Top %d Cities of your friends distributed' % len(cities), fontsize=title_font_size)
# 添加刻度标签
plt.yticks(range(len(cities)), cities)
# 设置X轴的刻度范围
plt.xlim([0, city_people[0] * 1.1])

# 为每个条形图添加数值标签
for x, y in enumerate(city_people):
  plt.text(y+5, x, y, ha='center')

# 显示图形
plt.savefig(result_path+'/4.png')
plt.show()

### 结果生成

In [0]:
!rm -rf how_many_friends_in_your_wechat/results/result.png
#!ls -al how_many_friends_in_your_wechat/results
img_path = result_path
imgs = []
[imgs.append(img_path+"/"+fn) for fn in os.listdir(img_path) if fn.endswith('.png')]
imgs.sort()

final_img = result_path + "/result.png"

# 生成summary
generate_summary(friends_num + friends_in_chatrooms, remarked_friends, star_friends)

# 生成最终结果
bk = combine_images(imgs, inner_img_spacing=60)

# 保存最终结果
bk.save(final_img)

# 将最终结果发到手机端「文件传输助手」
itchat.send_image(final_img, toUserName='filehelper')

print "已生成！请在手机端的「文件传输助手」查看结果，记得分享和关注猿湿Xoong喔！"
print "已生成！请在手机端的「文件传输助手」查看结果，记得分享和关注猿湿Xoong喔！"
print "已生成！请在手机端的「文件传输助手」查看结果，记得分享和关注猿湿Xoong喔！"